Copyright 2024 shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Vertex AI Agent Integration

Feedback : shins777@gmail.com.

* 이 Colab은 GCP Console 상에서 구성된 Vertex AI Agent를 API를 통해서 Integration 하는 예제를 설명합니다.
* 아래 예제는 https://cloud.google.com/python/docs/reference/dialogflow-cx/latest 를 참고해서 작성되었습니다.

#### 라이브러리 설치

In [ ]:
%pip install -q -U google-cloud-dialogflow-cx

#### Service Account를 통해서 인증 설정 : Credentials 생성

In [ ]:
# Project constant setting.
PROJECT_ID="ai-hangsik"
REGION="global"

# Create a creadential to authenticate an access to the GCP.
from google.oauth2 import service_account
import google.oauth2.credentials

# Location of service account. Use the service account having an IAM including Discovery engine access.
SVC_ACCOUNT_FILE = "/home/admin_/keys/ai-hangsik-71898c80c9a5.json"

credentials = service_account.Credentials.from_service_account_file(
    SVC_ACCOUNT_FILE,
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)

### Vertex AI Agent Integration
#### gRPC integration

아래 정보는 gRPC 형태로 연계하는 예제입니다. 예제는 아래 정보를 통해서 생성되었습니다.  
*  https://cloud.google.com/python/docs/reference/dialogflow-cx/latest
*  https://cloud.google.com/dialogflow/cx/docs/how/detect-intent-stream#sentiment

In [ ]:
import uuid
from google.cloud.dialogflowcx_v3 import AgentsClient
from google.cloud.dialogflowcx_v3 import SessionsClient
from google.cloud.dialogflowcx_v3.types import session

def request_agent(agent_id, session_id, texts ):
    """Returns the result of detect intent with texts as inputs.
    Using the same `session_id` between requests allows continuation
    of the conversation."""


    agent = f"projects/{PROJECT_ID}/locations/{REGION}/agents/{agent_id}"
    language_code = "en-us"

    session_path = f"{agent}/sessions/{session_id}"
    print(f"Session path: {session_path}\n")

    client_options = None
    agent_components = AgentsClient.parse_agent_path(agent)
    location_id = agent_components["location"]

    if location_id != "global":
        api_endpoint = f"{location_id}-dialogflow.googleapis.com:443"
        print(f"API Endpoint: {api_endpoint}\n")
        client_options = {"api_endpoint": api_endpoint}

    session_client = SessionsClient(client_options=client_options, credentials = credentials)

    for text in texts:
        text_input = session.TextInput(text=text)
        query_input = session.QueryInput(text=text_input, language_code=language_code)

        request = session.DetectIntentRequest(
            session=session_path, query_input=query_input
        )
        response = session_client.detect_intent(request=request)

        print("=" * 20)
        print(f"Query text: {response.query_result.text}")
        response_messages = [
            " ".join(msg.text.text) for msg in response.query_result.response_messages
        ]
        print(f"Response text: {' '.join(response_messages)}\n")


#### Request to agent with gRPC
*  agent id 는 Console에서 Agent 생성시 URL을 통해서 알 수 있습니다.
*  session id 는 여러 turn의 대화를 하나의 세션으로 처리하는 id 입니다.

In [ ]:
import time

t1 = time.time()

# text = "개인정보 보호법에 대해서 설명해주세요."
# text = "일반적인 질문으로 IT에서 PM의 역할은 무엇인가요?"
text = "정보주체의 권리에 대해서 설명해주세요."

agent_id = "ce0bf4ea-d2c2-46c4-b2a6-b6dd6c26ebbd"
session_id = uuid.uuid4()

texts = [text]

request_agent(agent_id, session_id, texts)

t2 = time.time()
print(f"latency : {t2-t1}")

### REST API Call to Vertex AI Agent.

#### REST API를 위한 Token 생성 테스트
아래 코드는 메뉴얼(https://cloud.google.com/dialogflow/vertex/docs/quick/api) 을 기반으로 작성되었습니다.

In [ ]:
# Use gcloud command.
!gcloud auth print-access-token

# Programtic method.
import os

stream = os.popen('gcloud auth print-access-token')
credential_token = stream.read().strip()
print(credential_token)

#### Request json file 생성.
이 파일은 REST API 호출시 사용됨.

In [ ]:
request = {
  "queryInput": {
    "text": {
      "text": "정보주체의 권리에 대해서 설명해주세요"
    },
    "languageCode": "en-us"
  },
  "queryParams": {
    "timeZone": "America/Los_Angeles"
  }
}

import json
with open('request.json', 'w') as fp:
    json.dump(request, fp)


In [ ]:
!curl -X POST \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    -H "x-goog-user-project: ai-hangsik" \
    -H "Content-Type: application/json; charset=utf-8" \
    -d @request.json \
    "https://global-dialogflow.googleapis.com/v3/projects/ai-hangsik/locations/global/agents/ce0bf4ea-d2c2-46c4-b2a6-b6dd6c26ebbd/sessions/9b9e818d-d61e-4bba-8836-3cc042a8d9fd:detectIntent"